In [1]:
import os
import sys
sys.path.insert(0, '/home/old-ufo/dev/caffe/python')
import caffe

from pylab import *

/home/old-ufo/dev/caffe/python/caffe/pycaffe.py:13: RuntimeWarning: to-Python converter for boost::shared_ptr<caffe::Net<float> > already registered; second conversion method ignored.
  from ._caffe import Net, SGDSolver, NesterovSolver, AdaGradSolver, \
/home/old-ufo/dev/caffe/python/caffe/pycaffe.py:13: RuntimeWarning: to-Python converter for boost::shared_ptr<caffe::Blob<float> > already registered; second conversion method ignored.
  from ._caffe import Net, SGDSolver, NesterovSolver, AdaGradSolver, \
/home/old-ufo/dev/caffe/python/caffe/pycaffe.py:13: RuntimeWarning: to-Python converter for boost::shared_ptr<caffe::Solver<float> > already registered; second conversion method ignored.
  from ._caffe import Net, SGDSolver, NesterovSolver, AdaGradSolver, \


In [2]:
import random
from caffe import layers as L
from caffe import params as P

                                            #Orthogonalization
import numpy as np

net = caffe.NetSpec()
namePerfix = ""
layersArray = []
n = caffe.NetSpec()
n.data, n.label = L.Data(batch_size=256, backend=P.Data.LMDB, source="",
                              ntop=2)
n.tops

imagenettraindb="/home/old-ufo/datasets/imagenet/ilsvrc12_train_shuffle_lmdb"
imagenetvaldb="/home/old-ufo/datasets/imagenet/ilsvrc12_val_lmdb"



We'll be running the provided LeNet example (make sure you've downloaded the data and created the databases, as below).

In [3]:
def addConvLayer(net,namePrefix,layersArray,channels=0,padding=0,kernel_size=0,
                 stride=1,group=1,explicit_name=None,weight_filler=dict(type='xavier'),bias_init=0):
    thisLayerName = namePrefix+"_conv"+str(len(layersArray))
    if(explicit_name != None):
        thisLayerName = explicit_name
    net.tops[thisLayerName] = L.Convolution(net.tops[layersArray[-1]], 
                                       kernel_size=kernel_size, 
                                       num_output=channels,
                                       pad=padding,
                                       stride=stride,
                                       group=group,
                                       param=[dict(decay_mult=1),dict(decay_mult=0)],
                                       weight_filler=weight_filler,
                                       bias_filler=dict(value=bias_init))
    layersArray.append(thisLayerName)
    return thisLayerName
def addEluLayer(net,namePrefix,layersArray,negative_slope=0, in_place=False, explicit_name=None):    
    thisLayerName = namePrefix+"_elu"+str(len(layersArray)-1)
    if(explicit_name != None):
        thisLayerName = explicit_name
    net.tops[thisLayerName] = L.ELU(net.tops[layersArray[-1]], in_place=in_place,
                                     elu_param=dict(alpha=negative_slope))
    if(not in_place):
        layersArray.append(thisLayerName)
    return thisLayerName    
def addReluLayer(net,namePrefix,layersArray,negative_slope=0, in_place=False, explicit_name=None):    
    thisLayerName = namePrefix+"_relu"+str(len(layersArray)-1)
    if(explicit_name != None):
        thisLayerName = explicit_name
    net.tops[thisLayerName] = L.ReLU(net.tops[layersArray[-1]], in_place=in_place,
                                     relu_param=dict(negative_slope=negative_slope))
    if(not in_place):
        layersArray.append(thisLayerName)
    return thisLayerName
def addConvRelu(net,namePrefix,layersArray,negative_slope=0.2,relu_in_place=False,
                channels=0,padding=0,kernel_size=0,stride=1,explicit_name=None):
    conv_explicit_name = None
    relu_explicit_name = None
    if(relu_in_place):
        conv_explicit_name = explicit_name
    else:
        relu_explicit_name = explicit_name
        
    addConvLayer(net,namePrefix,layersArray,channels=channels,padding=padding,
                 kernel_size=kernel_size,stride=stride,explicit_name=conv_explicit_name)
    return addReluLayer(net,namePrefix,layersArray,negative_slope=negative_slope,
                        in_place=relu_in_place,explicit_name=relu_explicit_name)
    
def addInnerProductLayer(net,namePrefix,layersArray,num_output = 0,explicit_name=None,bias_init = 0.0):
    thisLayerName = namePrefix+"_ip"+str(len(layersArray))
    if(explicit_name != None):
        thisLayerName = explicit_name
    net.tops[thisLayerName] = L.InnerProduct(net.tops[layersArray[-1]],
                                             num_output = num_output,
                                             param=[dict(decay_mult=1),
                                                    dict(decay_mult=BIAS_DECAY_MUL)],
                                            weight_filler=dict(type='gaussian', std=0.02),
                                            bias_filler=dict(value=bias_init))
    layersArray.append(thisLayerName)
    return thisLayerName
def addIPRelu(net,namePrefix,layersArray,negative_slope=0.2,
              relu_in_place=False,num_output=0,explicit_name=None):
    ip_explicit_name = None
    relu_explicit_name = None
    if(relu_in_place):
        ip_explicit_name = explicit_name
    else:
        relu_explicit_name = explicit_name
        
    addInnerProductLayer(net,namePrefix,layersArray,num_output=num_output,explicit_name=ip_explicit_name)
    return addReluLayer(net,namePrefix,layersArray,negative_slope=negative_slope,in_place=relu_in_place,explicit_name=relu_explicit_name)
 
def addSoftmaxWithLossLayer(net,namePrefix,layersArray,explicit_name=None,loss_weight=1):
    thisLayerName = namePrefix+"_loss"+str(len(layersArray))
    if(explicit_name != None):
        thisLayerName = explicit_name    
    net.tops[thisLayerName] = L.SoftmaxWithLoss(net.tops[layersArray[-2]],net.tops[layersArray[-1]],loss_weight=loss_weight)
    layersArray.append(thisLayerName)
    return thisLayerName
def addAccuracyLayer(net,namePrefix,layersArray,explicit_name=None):
    thisLayerName = namePrefix+"_accuracy"+str(len(layersArray))
    if(explicit_name != None):
        thisLayerName = explicit_name    
    net.tops[thisLayerName] = L.Accuracy(net.tops[layersArray[-2]],net.tops[layersArray[-1]])
    layersArray.append(thisLayerName)
    return thisLayerName

    

"""layer {
  name: "conv1_BN"
  type: "BatchNorm" include { phase: TRAIN}
  bottom: "conv1"
  top: "conv1_BN"
  param {
    lr_mult: 0
    decay_mult: 0
  }
  param {
    lr_mult: 0
    decay_mult: 0
  }
  param {
    lr_mult: 0
    decay_mult: 0
  }
  batch_norm_param {
    use_global_stats: false
    moving_average_fraction: 0.95
  }
}"""    
    
    
def addBatchNormLayer(net,namePrefix,layersArray,moving_average_fraction=0.95,
                      use_global_stats=False,explicit_name=None,in_place=False):
    thisLayerName = layersArray[-1]+"_BN"
    if(explicit_name != None):
        thisLayerName = explicit_name
    net.tops[thisLayerName] = L.BatchNorm(net.tops[layersArray[-1]],
                                          param = [dict(lr_mult=0,decay_mult=0)]*3,
                                          use_global_stats = use_global_stats,
                                          moving_average_fraction = moving_average_fraction
                                         )
    if(not in_place):
        layersArray.append(thisLayerName)
    return thisLayerName
    

def addScaleBiasLayer(net,namePrefix,layersArray,explicit_name=None,scale_init = 1.0,bias_init = 0.0,num_axes=1):
    thisLayerName = layersArray[-1]+"_scaled_biased"
    if(explicit_name != None):
        thisLayerName = explicit_name
    net.tops[thisLayerName] = L.Scale(net.tops[layersArray[-1]],
                                          bias_term=True,
                                          filler=dict(value=scale_init),
                                          bias_filler=dict(value=bias_init),
                                          num_axes=num_axes
                                         )
    layersArray.append(thisLayerName)    
    return thisLayerName
    
def addBiasLayer(net,namePrefix,layersArray,explicit_name=None,bias_init = 0.0,num_axes=1):
    thisLayerName = layersArray[-1]+"_biased"
    if(explicit_name != None):
        thisLayerName = explicit_name
    net.tops[thisLayerName] = L.Bias(net.tops[layersArray[-1]],
                                          filler=dict(value=bias_init),
                                          num_axes=num_axes
                                         )
    layersArray.append(thisLayerName)    
    return thisLayerName
    

def addDroupoutLayer(net,namePrefix,layersArray,dropout_ratio=0.5,explicit_name=None):
    thisLayerName = layersArray[-1]+"_dropout"
    if(explicit_name != None):
        thisLayerName = explicit_name
    net.tops[thisLayerName] = L.Dropout(net.tops[layersArray[-1]],
                                          dropout_ratio=dropout_ratio
                                         )
    layersArray.append(thisLayerName)    
    return thisLayerName

def addEltwiseLayer(net,namePrefix,layersArray,opeation=1,inputlayer1=None,inputlayer2=None, explicit_name=None):
    thisLayerName = layersArray[-1]+"_eltwise"
    if(explicit_name != None):
        thisLayerName = explicit_name
    net.tops[thisLayerName] = L.Eltwise(net.tops[layersArray[-1]],
                                          operation=1, bottom=[inputlayer1])
    layersArray.append(thisLayerName)    
    return thisLayerName
def addEALayer(net,namePrefix,layersArray,opeation=1,inputlayer1=None,inputlayer2=None, explicit_name=None):
    thisLayerName = layersArray[-1]+"_EA"
    if(explicit_name != None):
        thisLayerName = explicit_name
    net.tops[thisLayerName] = L.EltwiseAffine(net.tops[layersArray[-1]],
                                          slope_filler=dict(value=1.0),
                                          bias_filler=dict(value=0))
    layersArray.append(thisLayerName)    
    return thisLayerName
def addGlobalAvePoolingLayer(net,namePrefix,layersArray,explicit_name=None):
    thisLayerName = layersArray[-1]+"_global_ave"
    if(explicit_name != None):
        thisLayerName = explicit_name
        
    net.tops[thisLayerName] = L.Pooling(net.tops[layersArray[-1]],
                                          pooling_param = dict (
            pool = 1,global_pooling = True
        )
                                         )
    layersArray.append(thisLayerName)
    return thisLayerName

def addAvePoolingLayer(net,namePrefix,layersArray,kernel_size=1,stride=1,explicit_name=None):
    thisLayerName = layersArray[-1]+"_ave"
    if(explicit_name != None):
        thisLayerName = explicit_name
        
    net.tops[thisLayerName] = L.Pooling(net.tops[layersArray[-1]],
                                          pooling_param = dict (
            pool = 1,kernel_size = kernel_size,stride=stride
        )
                                         )
    layersArray.append(thisLayerName)
    return thisLayerName
def addMaxPoolingLayer(net,namePrefix,layersArray,kernel_size=1,stride=1,padding=0,explicit_name=None):
    thisLayerName = layersArray[-1]+"_max"
    if(explicit_name != None):
        thisLayerName = explicit_name
        
    net.tops[thisLayerName] = L.Pooling(net.tops[layersArray[-1]],
                                          pooling_param = dict (pool = 0,kernel_size = kernel_size,stride=stride,
                                                                pad=padding))
    layersArray.append(thisLayerName)
    return thisLayerName
def addFlatResBlock(net,namePrefix,layersArray,negative_slope=0.2,relu_in_place=False,
                channels=0,padding=0,kernel_size=0,stride=1,explicit_name=None):
    thisLayerName = namePrefix+"_FlatResBlock"+str(len(layersArray))
    conv1_name=layersArray[-1]
    conv_explicit_name = None
    relu_explicit_name = None
    if(relu_in_place):
        conv_explicit_name = explicit_name
    else:
        relu_explicit_name = explicit_name
        
    addConvLayer(net,namePrefix,layersArray,channels=channels,padding=padding,
                 kernel_size=kernel_size,stride=stride,explicit_name=conv_explicit_name)
    addBatchNormLayer(net,namePrefix,layersArray,moving_average_fraction=0.95,
                      use_global_stats=False,explicit_name=None)
    addReluLayer(net,namePrefix,layersArray,negative_slope=negative_slope,
                        in_place=relu_in_place,explicit_name=relu_explicit_name)
    
    addConvLayer(net,namePrefix,layersArray,channels=channels,padding=padding,
                 kernel_size=kernel_size,stride=1,explicit_name=conv_explicit_name)
    conv2_name=addBatchNormLayer(net,namePrefix,layersArray,moving_average_fraction=0.95,
                      use_global_stats=False,explicit_name=None)
    addEltwiseLayer(net,namePrefix,layersArray,opeation=1,inputlayer1=conv1_name,
                    inputlayer2=conv2_name, explicit_name=None)
    
    return addReluLayer(net,namePrefix,layersArray,negative_slope=negative_slope,
                        in_place=relu_in_place,explicit_name=relu_explicit_name)

def addBottleNeckResBlock(net,namePrefix,layersArray,negative_slope=0.2,relu_in_place=False,
                channels_wide=0,channels_narrow=0,padding=0,kernel_size=0,stride=1,explicit_name=None):
    thisLayerName = namePrefix+"_BottleNeckResBlock"+str(len(layersArray))
    conv1_name=layersArray[-1]
    conv_explicit_name = None
    relu_explicit_name = None
    if(relu_in_place):
        conv_explicit_name = explicit_name
    else:
        relu_explicit_name = explicit_name
        
    addConvLayer(net,namePrefix,layersArray,channels=channels_narrow,padding=0,
                 kernel_size=1,stride=stride,explicit_name=conv_explicit_name)
    addBatchNormLayer(net,namePrefix,layersArray,moving_average_fraction=0.95,
                      use_global_stats=False,explicit_name=None)
    addReluLayer(net,namePrefix,layersArray,negative_slope=negative_slope,
                        in_place=relu_in_place,explicit_name=relu_explicit_name)
    
    addConvLayer(net,namePrefix,layersArray,channels=channels_narrow,padding=padding,
                 kernel_size=kernel_size,stride=1,explicit_name=conv_explicit_name)
    addBatchNormLayer(net,namePrefix,layersArray,moving_average_fraction=0.95,
                      use_global_stats=False,explicit_name=None)
    addReluLayer(net,namePrefix,layersArray,negative_slope=negative_slope,
                        in_place=relu_in_place,explicit_name=relu_explicit_name)
    
    addConvLayer(net,namePrefix,layersArray,channels=channels_wide,padding=0,
                 kernel_size=1,stride=1,explicit_name=conv_explicit_name)
    conv2_name=addBatchNormLayer(net,namePrefix,layersArray,moving_average_fraction=0.95,
                      use_global_stats=False,explicit_name=None)
    
    addEltwiseLayer(net,namePrefix,layersArray,opeation=1,
                    inputlayer1=str(conv1_name), explicit_name=None)
    return addReluLayer(net,namePrefix,layersArray,negative_slope=negative_slope,
                        in_place=False,explicit_name=relu_explicit_name)
def addBottleNeckResBlockELUnoBN(net,namePrefix,layersArray,negative_slope=0.2,relu_in_place=False,
                channels_wide=0,channels_narrow=0,padding=0,kernel_size=0,stride=1,explicit_name=None):
    thisLayerName = namePrefix+"_BottleNeckResBlock"+str(len(layersArray))
    conv1_name=layersArray[-1]
    conv_explicit_name = None
    relu_explicit_name = None
    if(relu_in_place):
        conv_explicit_name = explicit_name
    else:
        relu_explicit_name = explicit_name
    alpha=1    
    addConvLayer(net,namePrefix,layersArray,channels=channels_narrow,padding=0,
                 kernel_size=1,stride=stride,explicit_name=conv_explicit_name)
    addEluLayer(net,namePrefix,layersArray,negative_slope=alpha,
                        in_place=relu_in_place,explicit_name=relu_explicit_name)
    
    addConvLayer(net,namePrefix,layersArray,channels=channels_narrow,padding=padding,
                 kernel_size=kernel_size,stride=1,explicit_name=conv_explicit_name)
    addEluLayer(net,namePrefix,layersArray,negative_slope=alpha,
                        in_place=relu_in_place,explicit_name=relu_explicit_name)
    
    conv2_name=addConvLayer(net,namePrefix,layersArray,channels=channels_wide,padding=0,
                 kernel_size=1,stride=1,explicit_name=conv_explicit_name)
    addEltwiseLayer(net,namePrefix,layersArray,opeation=1,
                    inputlayer1=str(conv1_name), explicit_name=None)
    return  addEluLayer(net,namePrefix,layersArray,negative_slope=alpha,
                        in_place=relu_in_place,explicit_name=relu_explicit_name)

def addBottleNeckResBlockNoBN(net,namePrefix,layersArray,negative_slope=0.2,relu_in_place=False,
                channels_wide=0,channels_narrow=0,padding=0,kernel_size=0,stride=1,explicit_name=None):
    thisLayerName = namePrefix+"_BottleNeckResBlock"+str(len(layersArray))
    conv1_name=layersArray[-1]
    conv_explicit_name = None
    relu_explicit_name = None
    if(relu_in_place):
        conv_explicit_name = explicit_name
    else:
        relu_explicit_name = explicit_name
        
    addConvLayer(net,namePrefix,layersArray,channels=channels_narrow,padding=0,
                 kernel_size=1,stride=stride,explicit_name=conv_explicit_name)
    #addBatchNormLayer(net,namePrefix,layersArray,moving_average_fraction=0.95,
    #                  use_global_stats=False,explicit_name=None)
    addReluLayer(net,namePrefix,layersArray,negative_slope=negative_slope,
                        in_place=relu_in_place,explicit_name=relu_explicit_name)
    
    addConvLayer(net,namePrefix,layersArray,channels=channels_narrow,padding=padding,
                 kernel_size=kernel_size,stride=1,explicit_name=conv_explicit_name)
    #addBatchNormLayer(net,namePrefix,layersArray,moving_average_fraction=0.95,
    #                  use_global_stats=False,explicit_name=None)
    addReluLayer(net,namePrefix,layersArray,negative_slope=negative_slope,
                        in_place=relu_in_place,explicit_name=relu_explicit_name)
    
    addConvLayer(net,namePrefix,layersArray,channels=channels_wide,padding=0,
                 kernel_size=1,stride=1,explicit_name=conv_explicit_name)
    #conv2_name=addBatchNormLayer(net,namePrefix,layersArray,moving_average_fraction=0.95,
    #                  use_global_stats=False,explicit_name=None)
    
    addEltwiseLayer(net,namePrefix,layersArray,opeation=1,
                    inputlayer1=str(conv1_name), explicit_name=None)
    return addReluLayer(net,namePrefix,layersArray,negative_slope=negative_slope,
                        in_place=False,explicit_name=relu_explicit_name)
def addBottleNeckResBlockBNEA(net,namePrefix,layersArray,negative_slope=0.2,relu_in_place=False,
                channels_wide=0,channels_narrow=0,padding=0,kernel_size=0,stride=1,explicit_name=None):
    thisLayerName = namePrefix+"_BottleNeckResBlock"+str(len(layersArray))
    conv1_name=layersArray[-1]
    conv_explicit_name = None
    relu_explicit_name = None
    if(relu_in_place):
        conv_explicit_name = explicit_name
    else:
        relu_explicit_name = explicit_name
        
    addConvLayer(net,namePrefix,layersArray,channels=channels_narrow,padding=0,
                 kernel_size=1,stride=stride,explicit_name=conv_explicit_name)
    addBatchNormLayer(net,namePrefix,layersArray,moving_average_fraction=0.95,
                      use_global_stats=False,explicit_name=None)
    addEALayer(net,namePrefix,layersArray,explicit_name=None)
    addReluLayer(net,namePrefix,layersArray,negative_slope=negative_slope,
                        in_place=relu_in_place,explicit_name=relu_explicit_name)
    
    addConvLayer(net,namePrefix,layersArray,channels=channels_narrow,padding=padding,
                 kernel_size=kernel_size,stride=1,explicit_name=conv_explicit_name)
    addBatchNormLayer(net,namePrefix,layersArray,moving_average_fraction=0.95,
                      use_global_stats=False,explicit_name=None)
    addEALayer(net,namePrefix,layersArray,explicit_name=None)
    addReluLayer(net,namePrefix,layersArray,negative_slope=negative_slope,
                        in_place=relu_in_place,explicit_name=relu_explicit_name)
    
    addConvLayer(net,namePrefix,layersArray,channels=channels_wide,padding=0,
                 kernel_size=1,stride=1,explicit_name=conv_explicit_name)
    addBatchNormLayer(net,namePrefix,layersArray,moving_average_fraction=0.95,
                      use_global_stats=False,explicit_name=None)
    addEALayer(net,namePrefix,layersArray,explicit_name=None)
    addEltwiseLayer(net,namePrefix,layersArray,opeation=1,
                    inputlayer1=str(conv1_name), explicit_name=None)
    return addReluLayer(net,namePrefix,layersArray,negative_slope=negative_slope,
                        in_place=False,explicit_name=relu_explicit_name)
def addBottleNeckResBlockBNScaleBias(net,namePrefix,layersArray,negative_slope=0.2,relu_in_place=False,
                channels_wide=0,channels_narrow=0,padding=0,kernel_size=0,stride=1,explicit_name=None):
    thisLayerName = namePrefix+"_BottleNeckResBlock"+str(len(layersArray))
    conv1_name=layersArray[-1]
    conv_explicit_name = None
    relu_explicit_name = None
    if(relu_in_place):
        conv_explicit_name = explicit_name
    else:
        relu_explicit_name = explicit_name
        
    addConvLayer(net,namePrefix,layersArray,channels=channels_narrow,padding=0,
                 kernel_size=1,stride=stride,explicit_name=conv_explicit_name)
    addBatchNormLayer(net,namePrefix,layersArray,moving_average_fraction=0.95,
                      use_global_stats=False,explicit_name=None)
    addScaleBiasLayer(net,namePrefix,layersArray,explicit_name=None,scale_init = 1.0,bias_init = 0.0,num_axes=1)
    addReluLayer(net,namePrefix,layersArray,negative_slope=negative_slope,
                        in_place=relu_in_place,explicit_name=relu_explicit_name)
    
    addConvLayer(net,namePrefix,layersArray,channels=channels_narrow,padding=padding,
                 kernel_size=kernel_size,stride=1,explicit_name=conv_explicit_name)
    addBatchNormLayer(net,namePrefix,layersArray,moving_average_fraction=0.95,
                      use_global_stats=False,explicit_name=None)
    addScaleBiasLayer(net,namePrefix,layersArray,explicit_name=None,scale_init = 1.0,bias_init = 0.0,num_axes=1)
    addReluLayer(net,namePrefix,layersArray,negative_slope=negative_slope,
                        in_place=relu_in_place,explicit_name=relu_explicit_name)
    
    addConvLayer(net,namePrefix,layersArray,channels=channels_wide,padding=0,
                 kernel_size=1,stride=1,explicit_name=conv_explicit_name)
    addBatchNormLayer(net,namePrefix,layersArray,moving_average_fraction=0.95,
                      use_global_stats=False,explicit_name=None)
    addScaleBiasLayer(net,namePrefix,layersArray,explicit_name=None,scale_init = 1.0,bias_init = 0.0,num_axes=1)
    addEltwiseLayer(net,namePrefix,layersArray,opeation=1,
                    inputlayer1=str(conv1_name), explicit_name=None)
    return addReluLayer(net,namePrefix,layersArray,negative_slope=negative_slope,
                        in_place=False,explicit_name=relu_explicit_name)

In [4]:
def ThinResNetNoBN(bs=256, db='db1'):
    net = caffe.NetSpec()
    layersArray=[];
    namePrefix='block1'
    
    net.tops["data"],net.tops["label"] = L.Data(batch_size=256, backend=P.Data.LMDB, source=db,  ntop=2)  
    layersArray=['data']
 #   net.tops["RandomDescriptor"] = L.DummyData(dummy_data_param = dict(
 #           shape = dict(dim=[bs,256]),
 #                        data_filler=dict(type="gaussian")))
 #   layersArray=["RandomDescriptor"]
    addConvLayer(net,namePrefix,layersArray,channels=64,padding=0,
                 kernel_size=7,stride=2)
 #   addBatchNormLayer(net,namePrefix,layersArray,moving_average_fraction=0.95,
 #                     use_global_stats=False,explicit_name=None)
    addReluLayer(net,namePrefix,layersArray,negative_slope=0,
                        in_place=True,explicit_name=None)
    addMaxPoolingLayer(net,namePrefix,layersArray,kernel_size=3,stride=2,explicit_name=None)
        #Block2
    namePrefix='block2'

    addConvLayer(net,namePrefix,layersArray,channels=256,padding=0,
             kernel_size=1,stride=2)
    addReluLayer(net,namePrefix,layersArray,negative_slope=0,
                        in_place=True,explicit_name=None)
    for i in range(3):
        addBottleNeckResBlock(net,namePrefix,layersArray,negative_slope=0,relu_in_place=True,
                              channels_wide=256,channels_narrow=64,padding=1,
                              kernel_size=3,stride=1,explicit_name=None);         
    namePrefix='block3'
    addConvLayer(net,namePrefix,layersArray,channels=512,padding=0,
             kernel_size=1,stride=2)
    addReluLayer(net,namePrefix,layersArray,negative_slope=0,
                        in_place=True,explicit_name=None)
    for i in range(2):
        addBottleNeckResBlock(net,namePrefix,layersArray,negative_slope=0,relu_in_place=True,
                              channels_wide=512,channels_narrow=128,padding=1,
                              kernel_size=3,stride=1,explicit_name=None);
    namePrefix='block4'
    addConvLayer(net,namePrefix,layersArray,channels=1024,padding=0,
             kernel_size=1,stride=2)
    addReluLayer(net,namePrefix,layersArray,negative_slope=0,
                        in_place=True,explicit_name=None)
    for i in range(23):
        addBottleNeckResBlock(net,namePrefix,layersArray,negative_slope=0,relu_in_place=True,
                              channels_wide=1024,channels_narrow=256,padding=1,
                              kernel_size=3,stride=1,explicit_name=None);

  #  addConvLayer(net,namePrefix,layersArray,channels=2048,padding=0,
  #           kernel_size=1,stride=2)
  #  addReluLayer(net,namePrefix,layersArray,negative_slope=0,
  #                      in_place=True,explicit_name=None)

    
  #  namePrefix='block5'
  #  for i in range(3):
  #      addBottleNeckResBlock(net,namePrefix,layersArray,negative_slope=0,relu_in_place=True,
  #                            channels_wide=2048,channels_narrow=512,padding=1,
  #                            kernel_size=3,stride=1,explicit_name=None);

    namePrefix='final'
    addConvLayer(net,namePrefix,layersArray,channels=1024,padding=0,
             kernel_size=1,stride=1)
  #  addBatchNormLayer(net,namePrefix,layersArray,moving_average_fraction=0.95,
  #                    use_global_stats=False,explicit_name=None)
  
    addReluLayer(net,namePrefix,layersArray,negative_slope=0,
                        in_place=True,explicit_name=None)
    addGlobalAvePoolingLayer(net,namePrefix,layersArray,explicit_name=None)
    addConvLayer(net,namePrefix,layersArray,channels=1000,padding=0,kernel_size=1,
                 stride=1,group=1,explicit_name=None,weight_filler=dict(type='xavier'),bias_init=0)
    clf_name=layersArray[-1];
    net.tops['loss'] = L.SoftmaxWithLoss(net.tops[clf_name],net.tops['label'],loss_weight=1)
    net.tops['acc'] =  L.Accuracy(net.tops[clf_name],net.tops['label'])
    
    return net.to_proto()

In [5]:
def ThinResNet(bs=256, db='db1'):
    net = caffe.NetSpec()
    layersArray=[];
    namePrefix='block1'
    
    net.tops["data"],net.tops["label"] = L.Data(batch_size=256, backend=P.Data.LMDB, source=db,  ntop=2)  
    layersArray=['data']
 #   net.tops["RandomDescriptor"] = L.DummyData(dummy_data_param = dict(
 #           shape = dict(dim=[bs,256]),
 #                        data_filler=dict(type="gaussian")))
 #   layersArray=["RandomDescriptor"]
    addConvLayer(net,namePrefix,layersArray,channels=64,padding=0,
                 kernel_size=7,stride=2)
    addBatchNormLayer(net,namePrefix,layersArray,moving_average_fraction=0.95,
                      use_global_stats=False,explicit_name=None)
    addReluLayer(net,namePrefix,layersArray,negative_slope=0,
                        in_place=True,explicit_name=None)
    addMaxPoolingLayer(net,namePrefix,layersArray,kernel_size=3,stride=2,explicit_name=None)
        #Block2
    namePrefix='block2'

    addConvLayer(net,namePrefix,layersArray,channels=256,padding=0,
             kernel_size=1,stride=2)
    addBatchNormLayer(net,namePrefix,layersArray,moving_average_fraction=0.95,
                      use_global_stats=False,explicit_name=None)
    addReluLayer(net,namePrefix,layersArray,negative_slope=0,
                        in_place=True,explicit_name=None)
    for i in range(3):
        addBottleNeckResBlock(net,namePrefix,layersArray,negative_slope=0,relu_in_place=True,
                              channels_wide=256,channels_narrow=64,padding=1,
                              kernel_size=3,stride=1,explicit_name=None);         
    namePrefix='block3'
    addConvLayer(net,namePrefix,layersArray,channels=512,padding=0,
             kernel_size=1,stride=2)
    addReluLayer(net,namePrefix,layersArray,negative_slope=0,
                        in_place=True,explicit_name=None)
    for i in range(2):
        addBottleNeckResBlock(net,namePrefix,layersArray,negative_slope=0,relu_in_place=True,
                              channels_wide=512,channels_narrow=128,padding=1,
                              kernel_size=3,stride=1,explicit_name=None);
    namePrefix='block4'
    addConvLayer(net,namePrefix,layersArray,channels=1024,padding=0,
             kernel_size=1,stride=2)
    addReluLayer(net,namePrefix,layersArray,negative_slope=0,
                        in_place=True,explicit_name=None)
    for i in range(23):
        addBottleNeckResBlock(net,namePrefix,layersArray,negative_slope=0,relu_in_place=True,
                              channels_wide=1024,channels_narrow=256,padding=1,
                              kernel_size=3,stride=1,explicit_name=None);

  #  addConvLayer(net,namePrefix,layersArray,channels=2048,padding=0,
  #           kernel_size=1,stride=2)
  #  addReluLayer(net,namePrefix,layersArray,negative_slope=0,
  #                      in_place=True,explicit_name=None)

    
  #  namePrefix='block5'
  #  for i in range(3):
  #      addBottleNeckResBlock(net,namePrefix,layersArray,negative_slope=0,relu_in_place=True,
  #                            channels_wide=2048,channels_narrow=512,padding=1,
  #                            kernel_size=3,stride=1,explicit_name=None);

    namePrefix='final'
    addConvLayer(net,namePrefix,layersArray,channels=1024,padding=0,
             kernel_size=1,stride=1)
    addBatchNormLayer(net,namePrefix,layersArray,moving_average_fraction=0.95,
                      use_global_stats=False,explicit_name=None)
  
    addReluLayer(net,namePrefix,layersArray,negative_slope=0,
                        in_place=True,explicit_name=None)
    addGlobalAvePoolingLayer(net,namePrefix,layersArray,explicit_name=None)
    addConvLayer(net,namePrefix,layersArray,channels=1000,padding=0,kernel_size=1,
                 stride=1,group=1,explicit_name=None,weight_filler=dict(type='xavier'),bias_init=0)
    clf_name=layersArray[-1];
    net.tops['loss'] = L.SoftmaxWithLoss(net.tops[clf_name],net.tops['label'],loss_weight=1)
    net.tops['acc'] =  L.Accuracy(net.tops[clf_name],net.tops['label'])
    
    return net.to_proto()

In [6]:
def ThinResNetBNEA(bs=256, db='db1'):
    net = caffe.NetSpec()
    layersArray=[];
    namePrefix='block1'
    
    net.tops["data"],net.tops["label"] = L.Data(batch_size=256, backend=P.Data.LMDB, source=db,  ntop=2)  
    layersArray=['data']
 #   net.tops["RandomDescriptor"] = L.DummyData(dummy_data_param = dict(
 #           shape = dict(dim=[bs,256]),
 #                        data_filler=dict(type="gaussian")))
 #   layersArray=["RandomDescriptor"]
    addConvLayer(net,namePrefix,layersArray,channels=64,padding=0,
                 kernel_size=7,stride=2)
    addBatchNormLayer(net,namePrefix,layersArray,moving_average_fraction=0.95,
                      use_global_stats=False,explicit_name=None)
    addEALayer(net,namePrefix,layersArray,explicit_name=None)
    addReluLayer(net,namePrefix,layersArray,negative_slope=0,
                        in_place=True,explicit_name=None)
    addMaxPoolingLayer(net,namePrefix,layersArray,kernel_size=3,stride=2,explicit_name=None)
        #Block2
    namePrefix='block2'

    addConvLayer(net,namePrefix,layersArray,channels=256,padding=0,
             kernel_size=1,stride=2)
    addBatchNormLayer(net,namePrefix,layersArray,moving_average_fraction=0.95,
                      use_global_stats=False,explicit_name=None)
    
    addEALayer(net,namePrefix,layersArray,explicit_name=None)
    addReluLayer(net,namePrefix,layersArray,negative_slope=0,
                        in_place=True,explicit_name=None)
    for i in range(3):
        addBottleNeckResBlockBNEA(net,namePrefix,layersArray,negative_slope=0,relu_in_place=True,
                              channels_wide=256,channels_narrow=64,padding=1,
                              kernel_size=3,stride=1,explicit_name=None);         
    namePrefix='block3'
    addConvLayer(net,namePrefix,layersArray,channels=512,padding=0,
             kernel_size=1,stride=2)
    addReluLayer(net,namePrefix,layersArray,negative_slope=0,
                        in_place=True,explicit_name=None)
    for i in range(2):
        addBottleNeckResBlockBNEA(net,namePrefix,layersArray,negative_slope=0,relu_in_place=True,
                              channels_wide=512,channels_narrow=128,padding=1,
                              kernel_size=3,stride=1,explicit_name=None);
    namePrefix='block4'
    addConvLayer(net,namePrefix,layersArray,channels=1024,padding=0,
             kernel_size=1,stride=2)
    addReluLayer(net,namePrefix,layersArray,negative_slope=0,
                        in_place=True,explicit_name=None)
    addBatchNormLayer(net,namePrefix,layersArray,moving_average_fraction=0.95,
                      use_global_stats=False,explicit_name=None)
    
    addEALayer(net,namePrefix,layersArray,explicit_name=None)
    for i in range(23):
        addBottleNeckResBlockBNEA(net,namePrefix,layersArray,negative_slope=0,relu_in_place=True,
                              channels_wide=1024,channels_narrow=256,padding=1,
                              kernel_size=3,stride=1,explicit_name=None);

  #  addConvLayer(net,namePrefix,layersArray,channels=2048,padding=0,
  #           kernel_size=1,stride=2)
  #  addReluLayer(net,namePrefix,layersArray,negative_slope=0,
  #                      in_place=True,explicit_name=None)

    
  #  namePrefix='block5'
  #  for i in range(3):
  #      addBottleNeckResBlock(net,namePrefix,layersArray,negative_slope=0,relu_in_place=True,
  #                            channels_wide=2048,channels_narrow=512,padding=1,
  #                            kernel_size=3,stride=1,explicit_name=None);

    namePrefix='final'
    addConvLayer(net,namePrefix,layersArray,channels=1024,padding=0,
             kernel_size=1,stride=1)
    addBatchNormLayer(net,namePrefix,layersArray,moving_average_fraction=0.95,
                      use_global_stats=False,explicit_name=None)
  
    addEALayer(net,namePrefix,layersArray,explicit_name=None)
    addReluLayer(net,namePrefix,layersArray,negative_slope=0,
                        in_place=True,explicit_name=None)
    addGlobalAvePoolingLayer(net,namePrefix,layersArray,explicit_name=None)
    addConvLayer(net,namePrefix,layersArray,channels=1000,padding=0,kernel_size=1,
                 stride=1,group=1,explicit_name=None,weight_filler=dict(type='xavier'),bias_init=0)
    clf_name=layersArray[-1];
    net.tops['loss'] = L.SoftmaxWithLoss(net.tops[clf_name],net.tops['label'],loss_weight=1)
    net.tops['acc'] =  L.Accuracy(net.tops[clf_name],net.tops['label'])
    
    return net.to_proto()

In [7]:
def ThinResNet50(bs=256, db='db1'):
    net = caffe.NetSpec()
    layersArray=[];
    namePrefix='block1'
    
    net.tops["data"],net.tops["label"] = L.Data(batch_size=256, backend=P.Data.LMDB, source=db,  ntop=2)  
    layersArray=['data']
 #   net.tops["RandomDescriptor"] = L.DummyData(dummy_data_param = dict(
 #           shape = dict(dim=[bs,256]),
 #                        data_filler=dict(type="gaussian")))
 #   layersArray=["RandomDescriptor"]
    addConvLayer(net,namePrefix,layersArray,channels=64,padding=0,
                 kernel_size=7,stride=2)
    addBatchNormLayer(net,namePrefix,layersArray,moving_average_fraction=0.98,
                      use_global_stats=False,explicit_name=None,in_place=True)
    addReluLayer(net,namePrefix,layersArray,negative_slope=0,
                        in_place=True,explicit_name=None)
    addMaxPoolingLayer(net,namePrefix,layersArray,kernel_size=3,stride=2,explicit_name=None)
    
    addConvLayer(net,namePrefix,layersArray,channels=256,padding=0,
                 kernel_size=1,stride=2)
        #Block2
    namePrefix='block2'
    for i in range(3):
        addBottleNeckResBlock(net,namePrefix,layersArray,negative_slope=0,relu_in_place=True,
                              channels_wide=256,channels_narrow=64,padding=1,
                              kernel_size=3,stride=1,explicit_name=None,bn_in_place=True); 
    addConvLayer(net,namePrefix,layersArray,channels=512,padding=0,
                 kernel_size=1,stride=2)
    
    namePrefix='block3'
#    addConvLayer(net,namePrefix,layersArray,channels=512,padding=0,
#             kernel_size=1,stride=2)
#    addReluLayer(net,namePrefix,layersArray,negative_slope=0,
#                        in_place=True,explicit_name=None)
    for i in range(4):
        addBottleNeckResBlock(net,namePrefix,layersArray,negative_slope=0,relu_in_place=True,
                              channels_wide=512,channels_narrow=128,padding=1,
                              kernel_size=3,stride=1,explicit_name=None,bn_in_place=True);
    addConvLayer(net,namePrefix,layersArray,channels=1024,padding=0,
                 kernel_size=1,stride=2)
    
    namePrefix='block4'
#    addConvLayer(net,namePrefix,layersArray,channels=1024,padding=0,
#             kernel_size=1,stride=2)
#    addReluLayer(net,namePrefix,layersArray,negative_slope=0,
#                        in_place=True,explicit_name=None)

    for i in range(6):
        addBottleNeckResBlock(net,namePrefix,layersArray,negative_slope=0,relu_in_place=True,
                              channels_wide=1024,channels_narrow=256,padding=1,
                              kernel_size=3,stride=1,explicit_name=None,bn_in_place=True);
    addConvLayer(net,namePrefix,layersArray,channels=2048,padding=0,
                 kernel_size=1,stride=2)
    
  #  addConvLayer(net,namePrefix,layersArray,channels=2048,padding=0,
  #           kernel_size=1,stride=2)
  #  addReluLayer(net,namePrefix,layersArray,negative_slope=0,
  #                      in_place=True,explicit_name=None)

    
    namePrefix='block5'
    for i in range(3):
        addBottleNeckResBlock(net,namePrefix,layersArray,negative_slope=0,relu_in_place=True,
                              channels_wide=2048,channels_narrow=512,padding=1,
                             kernel_size=3,stride=1,explicit_name=None,bn_in_place=True);

    namePrefix='final'
    addGlobalAvePoolingLayer(net,namePrefix,layersArray,explicit_name=None)
    addConvLayer(net,namePrefix,layersArray,channels=1000,padding=0,kernel_size=1,
                 stride=1,group=1,explicit_name=None,weight_filler=dict(type='xavier'),bias_init=0)
    clf_name=layersArray[-1];
    net.tops['loss'] = L.SoftmaxWithLoss(net.tops[clf_name],net.tops['label'],loss_weight=1)
    net.tops['acc'] =  L.Accuracy(net.tops[clf_name],net.tops['label'])
    
    return net.to_proto()

In [8]:
def ThinResNet50BNAfter(bs=256, db='db1'):
    net = caffe.NetSpec()
    layersArray=[];
    namePrefix='block1'
    
    net.tops["data"],net.tops["label"] = L.Data(batch_size=256, backend=P.Data.LMDB, source=db,  ntop=2)  
    layersArray=['data']
 #   net.tops["RandomDescriptor"] = L.DummyData(dummy_data_param = dict(
 #           shape = dict(dim=[bs,256]),
 #                        data_filler=dict(type="gaussian")))
 #   layersArray=["RandomDescriptor"]
    addConvLayer(net,namePrefix,layersArray,channels=64,padding=0,
                 kernel_size=7,stride=2)
    addReluLayer(net,namePrefix,layersArray,negative_slope=0,
                        in_place=True,explicit_name=None)
    addBatchNormLayer(net,namePrefix,layersArray,moving_average_fraction=0.98,
                      use_global_stats=False,explicit_name=None,in_place=True)
    addMaxPoolingLayer(net,namePrefix,layersArray,kernel_size=3,stride=2,explicit_name=None)
    
    addConvLayer(net,namePrefix,layersArray,channels=256,padding=0,
                 kernel_size=1,stride=1)
        #Block2
    namePrefix='block2'
    for i in range(3):
        addBottleNeckResBlockBNAfter1(net,namePrefix,layersArray,negative_slope=0,relu_in_place=True,
                              channels_wide=256,channels_narrow=64,padding=1,
                              kernel_size=3,stride=1,explicit_name=None,bn_in_place=True); 
    addConvLayer(net,namePrefix,layersArray,channels=512,padding=0,
                 kernel_size=1,stride=2)
    
    namePrefix='block3'
#    addConvLayer(net,namePrefix,layersArray,channels=512,padding=0,
#             kernel_size=1,stride=2)
#    addReluLayer(net,namePrefix,layersArray,negative_slope=0,
#                        in_place=True,explicit_name=None)
    for i in range(4):
        addBottleNeckResBlockBNAfter1(net,namePrefix,layersArray,negative_slope=0,relu_in_place=True,
                              channels_wide=512,channels_narrow=128,padding=1,
                              kernel_size=3,stride=1,explicit_name=None,bn_in_place=True);
    addConvLayer(net,namePrefix,layersArray,channels=1024,padding=0,
                 kernel_size=1,stride=2)
    
    namePrefix='block4'
#    addConvLayer(net,namePrefix,layersArray,channels=1024,padding=0,
#             kernel_size=1,stride=2)
#    addReluLayer(net,namePrefix,layersArray,negative_slope=0,
#                        in_place=True,explicit_name=None)

    for i in range(6):
        addBottleNeckResBlockBNAfter1(net,namePrefix,layersArray,negative_slope=0,relu_in_place=True,
                              channels_wide=1024,channels_narrow=256,padding=1,
                              kernel_size=3,stride=1,explicit_name=None,bn_in_place=True);
    addConvLayer(net,namePrefix,layersArray,channels=2048,padding=0,
                 kernel_size=1,stride=2)
    
  #  addConvLayer(net,namePrefix,layersArray,channels=2048,padding=0,
  #           kernel_size=1,stride=2)
  #  addReluLayer(net,namePrefix,layersArray,negative_slope=0,
  #                      in_place=True,explicit_name=None)

    
    namePrefix='block5'
    for i in range(3):
        addBottleNeckResBlockBNAfter1(net,namePrefix,layersArray,negative_slope=0,relu_in_place=True,
                              channels_wide=2048,channels_narrow=512,padding=1,
                             kernel_size=3,stride=1,explicit_name=None,bn_in_place=True);

    namePrefix='final'
    addGlobalAvePoolingLayer(net,namePrefix,layersArray,explicit_name=None)
    addConvLayer(net,namePrefix,layersArray,channels=1000,padding=0,kernel_size=1,
                 stride=1,group=1,explicit_name=None,weight_filler=dict(type='xavier'),bias_init=0)
    clf_name=layersArray[-1];
    net.tops['loss'] = L.SoftmaxWithLoss(net.tops[clf_name],net.tops['label'],loss_weight=1)
    net.tops['acc'] =  L.Accuracy(net.tops[clf_name],net.tops['label'])
    
    return net.to_proto()

In [9]:
def ThinResNet50BNAfter2(bs=256, db='db1'):
    net = caffe.NetSpec()
    layersArray=[];
    namePrefix='block1'
    
    net.tops["data"],net.tops["label"] = L.Data(batch_size=256, backend=P.Data.LMDB, source=db,  ntop=2)  
    layersArray=['data']
 #   net.tops["RandomDescriptor"] = L.DummyData(dummy_data_param = dict(
 #           shape = dict(dim=[bs,256]),
 #                        data_filler=dict(type="gaussian")))
 #   layersArray=["RandomDescriptor"]
    addConvLayer(net,namePrefix,layersArray,channels=64,padding=0,
                 kernel_size=7,stride=2)
    addReluLayer(net,namePrefix,layersArray,negative_slope=0,
                        in_place=True,explicit_name=None)
    addBatchNormLayer(net,namePrefix,layersArray,moving_average_fraction=0.98,
                      use_global_stats=False,explicit_name=None,in_place=True)
    addMaxPoolingLayer(net,namePrefix,layersArray,kernel_size=3,stride=2,explicit_name=None)
    
    addConvLayer(net,namePrefix,layersArray,channels=256,padding=0,
                 kernel_size=1,stride=2)
        #Block2
    namePrefix='block2'
    for i in range(3):
        addBottleNeckResBlockBNAfter2(net,namePrefix,layersArray,negative_slope=0,relu_in_place=True,
                              channels_wide=256,channels_narrow=64,padding=1,
                              kernel_size=3,stride=1,explicit_name=None,bn_in_place=True); 
    addConvLayer(net,namePrefix,layersArray,channels=512,padding=0,
                 kernel_size=1,stride=2)
    
    namePrefix='block3'
#    addConvLayer(net,namePrefix,layersArray,channels=512,padding=0,
#             kernel_size=1,stride=2)
#    addReluLayer(net,namePrefix,layersArray,negative_slope=0,
#                        in_place=True,explicit_name=None)
    for i in range(4):
        addBottleNeckResBlockBNAfter2(net,namePrefix,layersArray,negative_slope=0,relu_in_place=True,
                              channels_wide=512,channels_narrow=128,padding=1,
                              kernel_size=3,stride=1,explicit_name=None,bn_in_place=True);
    addConvLayer(net,namePrefix,layersArray,channels=1024,padding=0,
                 kernel_size=1,stride=2)
    
    namePrefix='block4'
#    addConvLayer(net,namePrefix,layersArray,channels=1024,padding=0,
#             kernel_size=1,stride=2)
#    addReluLayer(net,namePrefix,layersArray,negative_slope=0,
#                        in_place=True,explicit_name=None)

    for i in range(6):
        addBottleNeckResBlockBNAfter2(net,namePrefix,layersArray,negative_slope=0,relu_in_place=True,
                              channels_wide=1024,channels_narrow=256,padding=1,
                              kernel_size=3,stride=1,explicit_name=None,bn_in_place=True);
    addConvLayer(net,namePrefix,layersArray,channels=2048,padding=0,
                 kernel_size=1,stride=2)
    
  #  addConvLayer(net,namePrefix,layersArray,channels=2048,padding=0,
  #           kernel_size=1,stride=2)
  #  addReluLayer(net,namePrefix,layersArray,negative_slope=0,
  #                      in_place=True,explicit_name=None)

    
    namePrefix='block5'
    for i in range(3):
        addBottleNeckResBlockBNAfter2(net,namePrefix,layersArray,negative_slope=0,relu_in_place=True,
                              channels_wide=2048,channels_narrow=512,padding=1,
                             kernel_size=3,stride=1,explicit_name=None,bn_in_place=True);

    namePrefix='final'
    addGlobalAvePoolingLayer(net,namePrefix,layersArray,explicit_name=None)
    addConvLayer(net,namePrefix,layersArray,channels=1000,padding=0,kernel_size=1,
                 stride=1,group=1,explicit_name=None,weight_filler=dict(type='xavier'),bias_init=0)
    clf_name=layersArray[-1];
    net.tops['loss'] = L.SoftmaxWithLoss(net.tops[clf_name],net.tops['label'],loss_weight=1)
    net.tops['acc'] =  L.Accuracy(net.tops[clf_name],net.tops['label'])
    
    return net.to_proto()

In [10]:
def ThinResNet50ELUnoBN(bs=256, db='db1'):
    net = caffe.NetSpec()
    layersArray=[];
    namePrefix='block1'
    
    net.tops["data"],net.tops["label"] = L.Data(batch_size=256, backend=P.Data.LMDB, source=db,  ntop=2)  
    layersArray=['data']
 #   net.tops["RandomDescriptor"] = L.DummyData(dummy_data_param = dict(
 #           shape = dict(dim=[bs,256]),
 #                        data_filler=dict(type="gaussian")))
 #   layersArray=["RandomDescriptor"]
    addConvLayer(net,namePrefix,layersArray,channels=64,padding=0,
                 kernel_size=7,stride=2)
    addEluLayer(net,namePrefix,layersArray,negative_slope=1,
                        in_place=True,explicit_name=None)
    addMaxPoolingLayer(net,namePrefix,layersArray,kernel_size=3,stride=2, padding=1, explicit_name=None)
    addConvLayer(net,namePrefix,layersArray,channels=256,padding=0,
             kernel_size=1,stride=2)
        #Block2
    namePrefix='block2'
    for i in range(3):
        addBottleNeckResBlockELUnoBN(net,namePrefix,layersArray,negative_slope=0,relu_in_place=True,
                              channels_wide=256,channels_narrow=64,padding=1,
                              kernel_size=3,stride=1,explicit_name=None); 
    addConvLayer(net,namePrefix,layersArray,channels=512,padding=0,
                 kernel_size=1,stride=2)
    
    namePrefix='block3'
#    addConvLayer(net,namePrefix,layersArray,channels=512,padding=0,
#             kernel_size=1,stride=2)
#    addReluLayer(net,namePrefix,layersArray,negative_slope=0,
#                        in_place=True,explicit_name=None)
    for i in range(4):
        addBottleNeckResBlockELUnoBN(net,namePrefix,layersArray,negative_slope=0,relu_in_place=True,
                              channels_wide=512,channels_narrow=128,padding=1,
                              kernel_size=3,stride=1,explicit_name=None);
    addConvLayer(net,namePrefix,layersArray,channels=1024,padding=0,
                 kernel_size=1,stride=2)
    
    namePrefix='block4'
    
    for i in range(6):
        addBottleNeckResBlockELUnoBN(net,namePrefix,layersArray,negative_slope=0,relu_in_place=True,
                              channels_wide=1024,channels_narrow=256,padding=1,
                              kernel_size=3,stride=1,explicit_name=None);
    addConvLayer(net,namePrefix,layersArray,channels=2048,padding=0,
                 kernel_size=1,stride=2)
    
    namePrefix='block5'
    for i in range(3):
        addBottleNeckResBlockELUnoBN(net,namePrefix,layersArray,negative_slope=0,relu_in_place=True,
                              channels_wide=2048,channels_narrow=512,padding=1,
                             kernel_size=3,stride=1,explicit_name=None);

    namePrefix='final'
    addGlobalAvePoolingLayer(net,namePrefix,layersArray,explicit_name=None)
    addConvLayer(net,namePrefix,layersArray,channels=1000,padding=0,kernel_size=1,
                 stride=1,group=1,explicit_name=None,weight_filler=dict(type='xavier'),bias_init=0)
    clf_name=layersArray[-1];
    net.tops['loss'] = L.SoftmaxWithLoss(net.tops[clf_name],net.tops['label'],loss_weight=1)
    net.tops['acc'] =  L.Accuracy(net.tops[clf_name],net.tops['label'])
    
    return net.to_proto()

In [11]:
def ThinResNet50ReLUBN(bs=256, db='db1'):
    net = caffe.NetSpec()
    layersArray=[];
    namePrefix='block1'
    
    net.tops["data"],net.tops["label"] = L.Data(batch_size=256, backend=P.Data.LMDB, source=db,  ntop=2)  
    layersArray=['data']
 #   net.tops["RandomDescriptor"] = L.DummyData(dummy_data_param = dict(
 #           shape = dict(dim=[bs,256]),
 #                        data_filler=dict(type="gaussian")))
 #   layersArray=["RandomDescriptor"]
    addConvLayer(net,namePrefix,layersArray,channels=64,padding=0,
                 kernel_size=7,stride=2)
    addBatchNormLayer(net,namePrefix,layersArray,moving_average_fraction=0.98,
                      use_global_stats=False,explicit_name=None,in_place=True)
    addScaleBiasLayer(net,namePrefix,layersArray,explicit_name=None,scale_init = 1.0,bias_init = 0.0,num_axes=1)
    addReluLayer(net,namePrefix,layersArray,negative_slope=0,
                        in_place=True,explicit_name=None)
    addMaxPoolingLayer(net,namePrefix,layersArray,kernel_size=3,stride=2, padding=1, explicit_name=None)
    addConvLayer(net,namePrefix,layersArray,channels=256,padding=0,
             kernel_size=1,stride=2)
       #Block2
    namePrefix='block2'
    for i in range(3):
        addBottleNeckResBlockBNScaleBias(net,namePrefix,layersArray,negative_slope=0,relu_in_place=True,
                              channels_wide=256,channels_narrow=64,padding=1,
                              kernel_size=3,stride=1,explicit_name=None); 
    addConvLayer(net,namePrefix,layersArray,channels=512,padding=0,
                 kernel_size=1,stride=2)
    
    namePrefix='block3'
#    addConvLayer(net,namePrefix,layersArray,channels=512,padding=0,
#             kernel_size=1,stride=2)
#    addReluLayer(net,namePrefix,layersArray,negative_slope=0,
#                        in_place=True,explicit_name=None)
    for i in range(4):
        addBottleNeckResBlockBNScaleBias(net,namePrefix,layersArray,negative_slope=0,relu_in_place=True,
                              channels_wide=512,channels_narrow=128,padding=1,
                              kernel_size=3,stride=1,explicit_name=None);
    addConvLayer(net,namePrefix,layersArray,channels=1024,padding=0,
                 kernel_size=1,stride=2)
    
    namePrefix='block4'
    
    for i in range(6):
        addBottleNeckResBlockBNScaleBias(net,namePrefix,layersArray,negative_slope=0,relu_in_place=True,
                              channels_wide=1024,channels_narrow=256,padding=1,
                              kernel_size=3,stride=1,explicit_name=None);
    addConvLayer(net,namePrefix,layersArray,channels=2048,padding=0,
                 kernel_size=1,stride=2)
    
    namePrefix='block5'
    for i in range(3):
        addBottleNeckResBlockBNScaleBias(net,namePrefix,layersArray,negative_slope=0,relu_in_place=True,
                              channels_wide=2048,channels_narrow=512,padding=1,
                             kernel_size=3,stride=1,explicit_name=None);

    namePrefix='final'
    addGlobalAvePoolingLayer(net,namePrefix,layersArray,explicit_name=None)
    addConvLayer(net,namePrefix,layersArray,channels=1000,padding=0,kernel_size=1,
                 stride=1,group=1,explicit_name=None,weight_filler=dict(type='xavier'),bias_init=0)
    clf_name=layersArray[-1];
    net.tops['loss'] = L.SoftmaxWithLoss(net.tops[clf_name],net.tops['label'],loss_weight=1)
    net.tops['acc'] =  L.Accuracy(net.tops[clf_name],net.tops['label'])
    
    return net.to_proto()

In [12]:

with open('resnetbn_50_bn_after_sb1.prototxt', 'w') as f:
    f.write(str( ThinResNet50ReLUBN(256,imagenettraindb)))
#nn=ThinResNet(256,imagenettraindb)